In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import seaborn as sns

#load tables
customers = pd.read_csv("customers.csv")
products = pd.read_csv("products.csv")
stores = pd.read_csv("stores.csv")
employees = pd.read_csv("employees.csv")
discounts = pd.read_csv("discounts.csv")
transactions = pd.read_csv("transactions.csv")

In [ ]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 663870 entries, 0 to 663869
Data columns (total 20 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Transaction_ID    663870 non-null  int64  
 1   Invoice_ID        663870 non-null  object 
 2   Line              663870 non-null  int64  
 3   Customer_ID       663870 non-null  int64  
 4   Product_ID        663870 non-null  int64  
 5   Size              621115 non-null  object 
 6   Color             217232 non-null  object 
 7   Unit_Price        663870 non-null  float64
 8   Quantity          663870 non-null  int64  
 9   Date              663870 non-null  object 
 10  Discount          663870 non-null  float64
 11  Line_Total        663870 non-null  float64
 12  Store_ID          663870 non-null  int64  
 13  Employee_ID       663870 non-null  int64  
 14  Currency          663870 non-null  object 
 15  Currency_Symbol   663870 non-null  object 
 16  SKU               66

In [ ]:
# BASIC SHAPE CHECKS
print(f"Transactions: {transactions.shape}, Customers: {customers.shape}, Products: {products.shape}, Stores: {stores.shape}")

# NULL CHECKS
for name, df in [('Transactions', transactions), ('Customers', customers),
                 ('Products', products), ('Stores', stores)]:
    missing = df.isna().sum().sum()
    print(f"{name}: {missing} missing values total")

# DUPLICATE CHECKS
print(f"Duplicate Transactions: {transactions.duplicated().sum()}")

# KEY COLUMN VALIDATION
print("🔑 Key Column Integrity Check")
missing_customers = transactions[~transactions['Customer_ID'].isin(customers['Customer_ID'])]
missing_products = transactions[~transactions['Product_ID'].isin(products['Product_ID'])]
print(f"Missing Customer IDs in transactions: {len(missing_customers)}")
print(f"Missing Product IDs in transactions: {len(missing_products)}")

Transactions: (6284272, 20), Customers: (1643306, 9), Products: (14950, 13), Stores: (35, 8)
Transactions: 4664857 missing values total
Customers: 584153 missing values total
Products: 12107 missing values total
Stores: 0 missing values total
Duplicate Transactions: 0
🔑 Key Column Integrity Check
Missing Customer IDs in transactions: 0
Missing Product IDs in transactions: 0


In [ ]:
# Row counts
for name, df in [("Customers", customers), ("Products", products),
                 ("Employees", employees), ("Stores", stores),
                 ("Discounts", discounts), ("Transactions", transactions)]:
    print(f"{name}: {df.shape[0]} rows, {df.shape[1]} cols")

# Duplicates check
print("Duplicate Customers:", customers["Customer_ID"].duplicated().sum())
print("Duplicate Products:", products["Product_ID"].duplicated().sum())
print("Duplicate Employees:", employees["Employee_ID"].duplicated().sum())

Customers: 1643306 rows, 9 cols
Products: 14950 rows, 13 cols
Employees: 403 rows, 4 cols
Stores: 35 rows, 8 cols
Discounts: 204 rows, 6 cols
Transactions: 6284272 rows, 20 cols
Duplicate Customers: 0
Duplicate Products: 0
Duplicate Employees: 0


In [ ]:
# ✅ Verify foreign key integrity before joins

# transactions, customers, products, stores, employees, discounts

# Define expected foreign key relationships
fk_checks = {
    "Customer_ID": customers["Customer_ID"],
    "Product_ID": products["Product_ID"],
    "Store_ID": stores["Store_ID"],
    "Employee_ID": employees["Employee_ID"],
    # "Discount_ID": discounts["Discount_ID"], # Removed as Discount_ID column does not exist
}

for col, valid_ids in fk_checks.items():
    if col in transactions.columns:
        missing = ~transactions[col].isin(valid_ids)
        missing_count = missing.sum()
        total = len(transactions)
        print(f"{col}: {missing_count} missing ({(missing_count/total)*100:.2f}%)")
        if missing_count > 0:
            print(f"⚠️  Example missing IDs: {transactions.loc[missing, col].unique()[:5]}")
    else:
        print(f"{col}: not found in Transactions table.")

Customer_ID: 0 missing (0.00%)
Product_ID: 0 missing (0.00%)
Store_ID: 0 missing (0.00%)
Employee_ID: 0 missing (0.00%)


In [ ]:
# Prepare a mapping: column in transactions -> valid IDs series in lookup table
fk_checks = {
    "Customer_ID": customers["Customer_ID"] if "Customer_ID" in customers.columns else None,
    "Product_ID": products["Product_ID"] if "Product_ID" in products.columns else None,
    "Store_ID": stores["Store_ID"] if "Store_ID" in stores.columns else None,
    "Employee_ID": employees["Employee_ID"] if "Employee_ID" in employees.columns else None,
    # Do not include Discount_ID if it doesn't exist
}

print("🔎 Running FK checks (only for keys that exist in both sides)...\n")
for tx_col, valid_ids in fk_checks.items():
    if tx_col not in transactions.columns:
        print(f"• {tx_col}: not present in transactions → skipping")
        continue
    if valid_ids is None:
        print(f"• {tx_col}: lookup table missing column → skipping")
        continue

    missing_mask = ~transactions[tx_col].isin(valid_ids)
    missing_count = int(missing_mask.sum())
    total = len(transactions)
    print(f"• {tx_col}: {missing_count:,} missing ({missing_count/total*100:.4f}%)")
    if missing_count:
        print(f"   Example missing values (up to 10): {transactions.loc[missing_mask, tx_col].unique()[:10]}")
        # optionally save a sample for inspection
        sample = transactions.loc[missing_mask].head(5)
        display(sample)

# ----------------------------
# Discount-specific checks
# ----------------------------
print("\n🔎 Discount-related checks")

# 1) Is there an explicit Discount_ID link in either table?
if "Discount_ID" in transactions.columns and "Discount_ID" in discounts.columns:
    missing_mask = ~transactions["Discount_ID"].isin(discounts["Discount_ID"])
    print(f"• Discount_ID FK missing: {missing_mask.sum():,} rows")
else:
    print("• No Discount_ID FK available in both tables. Checking alternative discount columns...")

# 2) If transactions have DiscountCode or DiscountName or DiscountPercent, inspect those
alt_discount_cols = [c for c in ("DiscountCode", "Discount_Name", "DiscountPercent", "Discount") if c in transactions.columns]
if alt_discount_cols:
    print(f"• Found discount-like columns in transactions: {alt_discount_cols}")
    # Quick checks: % of transactions with non-null discount info; distribution of DiscountPercent
    if "DiscountPercent" in transactions.columns:
        non_null = transactions["DiscountPercent"].notna().sum()
        print(f"  - DiscountPercent present for {non_null:,} / {len(transactions):,} transactions ({non_null/len(transactions):.2%})")
        print("  - DiscountPercent summary:")
        display(transactions["DiscountPercent"].describe().to_frame().T)
    # If there's a discounts table with codes, compare unique codes
    if "DiscountCode" in transactions.columns and "Code" in discounts.columns:
        tx_codes = set(transactions["DiscountCode"].dropna().unique())
        lookup_codes = set(discounts["Code"].dropna().unique())
        missing_codes = sorted(list(tx_codes - lookup_codes))[:10]
        print(f"  - Discount codes in transactions not found in discounts table: {len(tx_codes - lookup_codes)} (example: {missing_codes})")
else:
    print("• No discount-related columns found in transactions to check.")

# ----------------------------
# Optionally save orphan rows for offline inspection
# ----------------------------
save_orphans = False  # set True to save example orphan rows
if save_orphans:
    orphan_dir = "orphan_samples"
    import os
    os.makedirs(orphan_dir, exist_ok=True)
    for tx_col, valid_ids in fk_checks.items():
        if tx_col in transactions.columns and valid_ids is not None:
            missing_mask = ~transactions[tx_col].isin(valid_ids)
            if missing_mask.any():
                fname = f"{orphan_dir}/orphans_{tx_col}.csv"
                transactions.loc[missing_mask].head(100).to_csv(fname, index=False)
                print(f"Saved sample orphans for {tx_col} -> {fname}")

print("\n✅ FK checks complete.")


🔎 Running FK checks (only for keys that exist in both sides)...

• Customer_ID: 0 missing (0.0000%)
• Product_ID: 0 missing (0.0000%)
• Store_ID: 0 missing (0.0000%)
• Employee_ID: 0 missing (0.0000%)

🔎 Discount-related checks
• No Discount_ID FK available in both tables. Checking alternative discount columns...
• Found discount-like columns in transactions: ['Discount']

✅ FK checks complete.


In [ ]:
current_date = pd.Timestamp.today().normalize()

transactions["Date"] = pd.to_datetime(transactions["Date"], errors="coerce")
print("📅 Dataset time window:")
print(f"   Start: {transactions['Date'].min().date()}")
print(f"   End:   {transactions['Date'].max().date()}")
print(f"   Current date reference for churn: {current_date.date()}")

📅 Dataset time window:
   Start: 2023-01-01
   End:   2025-02-22
   Current date reference for churn: 2025-11-10


In [ ]:
# ================================================================
# 🌟 FULL FEATURE ENGINEERING PIPELINE (CHURN-FIXED)
# ================================================================

import pandas as pd

# ------------------------------
# 0️⃣ Helper: Optimize DataFrame
# ------------------------------
def optimize_df(df, name="df"):
    print(f"\nOptimizing {name}...")
    original_mem = df.memory_usage(deep=True).sum() / 1024**2

    # Convert object columns to category first
    for col in df.select_dtypes(include="object").columns:
        df[col] = df[col].astype("category")

    # Use convert_dtypes for the rest
    df = df.convert_dtypes()
    final_mem = df.memory_usage(deep=True).sum() / 1024**2
    print(f"✅ Optimized {name}: {df.shape[0]:,} rows, {final_mem:.2f} MB (from {original_mem:.2f} MB)")
    return df

# ------------------------------
# 1️⃣ Optimize base tables
# ------------------------------
transactions = optimize_df(transactions, "transactions")
customers = optimize_df(customers, "customers")
products = optimize_df(products, "products")
stores = optimize_df(stores, "stores")

# ------------------------------
# 2️⃣ Set reference date for churn
# ------------------------------
# Use dataset max date + small buffer to avoid 100% churn
current_date = transactions["Date"].max() + pd.Timedelta(days=30)
print(f"\n📅 Using reference date for churn: {current_date.date()}")

# ------------------------------
# 3️⃣ Customer Behavior (RFM + Lifecycle)
# ------------------------------
customer_behavior = transactions.groupby('Customer_ID').agg({
    'Invoice_ID': 'nunique',
    'Line_Total': ['sum','mean','std'],
    'Date': ['min','max','count'],
    'Product_ID': 'nunique',
    'Store_ID': 'nunique',
    'Discount': 'sum'
}).round(2)

customer_behavior.columns = [
    'total_orders','total_spent','avg_order_value','spending_std',
    'first_purchase','last_purchase','total_transactions',
    'unique_products_bought','unique_stores_visited','total_discount_used'
]

customer_behavior['days_since_last_purchase'] = (current_date - customer_behavior['last_purchase']).dt.days
customer_behavior['days_as_customer'] = (customer_behavior['last_purchase'] - customer_behavior['first_purchase']).dt.days + 1
customer_behavior['purchase_frequency_per_month'] = (
    customer_behavior['total_orders'] / customer_behavior['days_as_customer'] * 30
).round(2)

# RFM scoring
customer_behavior['recency_score'] = pd.qcut(customer_behavior['days_since_last_purchase'], 5, labels=[5,4,3,2,1], duplicates='drop')
customer_behavior['frequency_score'] = pd.qcut(customer_behavior['total_orders'].rank(method='first'), 5, labels=[1,2,3,4,5], duplicates='drop')
customer_behavior['monetary_score'] = pd.qcut(customer_behavior['total_spent'].rank(method='first'), 5, labels=[1,2,3,4,5], duplicates='drop')

# Lifecycle stage
customer_behavior['lifecycle_stage'] = 'Active'
customer_behavior.loc[customer_behavior['total_orders'] == 1, 'lifecycle_stage'] = 'New'
customer_behavior.loc[customer_behavior['days_since_last_purchase'] > 180, 'lifecycle_stage'] = 'At_Risk'
customer_behavior.loc[customer_behavior['days_since_last_purchase'] > 365, 'lifecycle_stage'] = 'Churned'

# Churn flag
customer_behavior['is_churned'] = customer_behavior['lifecycle_stage'].isin(['At_Risk','Churned']).astype(int)

# Value segment
customer_behavior['customer_value_segment'] = pd.qcut(customer_behavior['total_spent'].rank(method='first'),
                                                      3, labels=['Low_Value','Medium_Value','High_Value'],
                                                      duplicates='drop')

# ------------------------------
# 4️⃣ Extend with demographics
# ------------------------------
customers["Date_Of_Birth"] = pd.to_datetime(customers["Date_Of_Birth"], errors='coerce')
customers["Age"] = (current_date.year - customers["Date_Of_Birth"].dt.year).fillna(-1).astype(int)
customers["AgeGroup"] = pd.cut(customers["Age"], bins=[0,25,40,60,100], labels=["<25","25-40","40-60","60+"], right=False)

customer_features = customer_behavior.reset_index().merge(
    customers[["Customer_ID","Gender","Country","Age","AgeGroup"]],
    on="Customer_ID", how="left"
).set_index("Customer_ID")

# ------------------------------
# 5️⃣ Category Affinity
# ------------------------------
df_cat = transactions.merge(products[["Product_ID","Category"]], on="Product_ID", how="left")
cat_spend = df_cat.groupby(["Customer_ID","Category"])["Line_Total"].sum().unstack(fill_value=0)
cat_spend["total"] = cat_spend.sum(axis=1)
for col in ["Feminine","Masculine","Children"]:
    if col in cat_spend:
        cat_spend[f"pct_spend_{col.lower()}"] = cat_spend[col] / cat_spend["total"]
cat_spend["top_category"] = cat_spend[["Feminine","Masculine","Children"]].idxmax(axis=1)

customer_features = customer_features.join(cat_spend.filter(like="pct_spend"))
customer_features["top_category"] = cat_spend["top_category"]

# ------------------------------
# 6️⃣ Basket features
# ------------------------------
customer_features["avg_basket_size"] = transactions.groupby("Customer_ID")["Quantity"].mean()
customer_features["avg_basket_value"] = (customer_features["total_spent"] / customer_features["total_orders"]).round(2)
customer_features["basket_value_std"] = transactions.groupby("Customer_ID")["Line_Total"].std()

# ------------------------------
# 7️⃣ Discount sensitivity
# ------------------------------
discounted_orders = transactions[transactions["Discount"] > 0].groupby("Customer_ID")["Invoice_ID"].nunique()
customer_features["pct_discounted_orders"] = (discounted_orders / customer_features["total_orders"]).fillna(0)
customer_features["avg_discount_per_order"] = (customer_features["total_discount_used"] / customer_features["total_orders"]).round(2)

# ------------------------------
# 8️⃣ Engagement over time
# ------------------------------
customer_features["avg_inter_purchase_gap"] = (customer_features["days_as_customer"] / customer_features["total_orders"]).round(1)
customer_features["purchase_trend"] = ((customer_features["total_transactions"] / customer_features["days_as_customer"]) / customer_features["purchase_frequency_per_month"]).round(2)

# ------------------------------
# 9️⃣ Store loyalty
# ------------------------------
store_counts = transactions.groupby(["Customer_ID","Store_ID"])["Invoice_ID"].count().reset_index()
home_store = store_counts.loc[store_counts.groupby("Customer_ID")["Invoice_ID"].idxmax()]
customer_features["home_store"] = home_store.set_index("Customer_ID")["Store_ID"]
customer_features["store_diversity"] = (customer_features["unique_stores_visited"] / customer_features["total_orders"]).round(2)

# ------------------------------
# 1️⃣0️⃣ Save dataset
# ------------------------------
customer_features.to_csv("features_customers.csv", index=True)
print("\n✅ Customer features ready with churn target and all extended features!")
print(customer_features[['total_orders','total_spent','lifecycle_stage','is_churned']].head())



Optimizing transactions...
✅ Optimized transactions: 6,284,272 rows, 1147.85 MB (from 3249.16 MB)

Optimizing customers...
✅ Optimized customers: 1,643,306 rows, 434.97 MB (from 804.98 MB)

Optimizing products...
✅ Optimized products: 14,950 rows, 7.34 MB (from 11.33 MB)

Optimizing stores...
✅ Optimized stores: 35 rows, 0.01 MB (from 0.01 MB)

📅 Using reference date for churn: 2025-03-24

✅ Customer features ready with churn target and all extended features!
             total_orders  total_spent lifecycle_stage  is_churned
Customer_ID                                                       
1                       2       158.28         Churned           1
2                       5        763.0          Active           0
3                       4        276.0          Active           0
4                       5        177.0          Active           0
5                       5       194.75         At_Risk           1


In [ ]:
# Show all columns
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 300)  # optional, adjust width

# Display first few rows
print(customer_features.head())

             total_orders  total_spent  avg_order_value  spending_std      first_purchase       last_purchase  total_transactions  unique_products_bought  unique_stores_visited  total_discount_used  days_since_last_purchase  days_as_customer  purchase_frequency_per_month recency_score  \
Customer_ID                                                                                                                                                                                                                                                                                     
1                       2       158.28            39.57         32.21 2023-03-09 18:45:00 2023-09-01 17:05:00                   4                       4                      1                  0.9                       570               176                          0.34             1   
2                       5        763.0            69.36         39.94 2024-01-20 14:48:00 2025-02-16 12:38:00                  11    

In [ ]:
# PART 2: PRODUCT PERFORMANCE & CHARACTERISTICS

print("\n" + "="*70)
print(" PART 2: PRODUCT PERFORMANCE & CHARACTERISTICS")
print("="*70)

# 1. Sales performance

product_sales = transactions.groupby("Product_ID").agg({
    "Line_Total": ["sum", "mean", "count"],
    "Quantity": "sum",
    "Unit_Price": "mean",
    "Customer_ID": "nunique",
    "Store_ID": "nunique",
    "Date": ["min", "max"]
}).round(2)

product_sales.columns = [
    "total_revenue", "avg_revenue_per_txn", "num_transactions",
    "total_units_sold", "avg_unit_price", "unique_customers",
    "num_stores", "first_sale_date", "last_sale_date"
]

# Lifecycle
product_sales["days_on_market"] = (
    product_sales["last_sale_date"] - product_sales["first_sale_date"]
).dt.days + 1
product_sales["sales_velocity"] = (
    product_sales["num_transactions"] / product_sales["days_on_market"] * 30
).round(2)

# 2. Merge product details

products["Production_Cost"] = pd.to_numeric(products["Production_Cost"], errors="coerce")

product_features = (
    product_sales.reset_index()
    .merge(
        products[["Product_ID", "Category", "Sub_Category", "Color", "Production_Cost"]],
        on="Product_ID",
        how="left"
    )
)

# 3. Profitability

product_features["profit_per_unit"] = (
    product_features["avg_unit_price"] - product_features["Production_Cost"]
)
product_features["profit_margin_pct"] = (
    product_features["profit_per_unit"] / product_features["avg_unit_price"] * 100
).round(2)
product_features["total_profit"] = (
    product_features["profit_per_unit"] * product_features["total_units_sold"]
)

# 4. Positioning & Popularity

product_features["price_tier"] = pd.qcut(
    product_features["avg_unit_price"],
    q=3, labels=["Budget", "Mid_Range", "Premium"], duplicates="drop"
)

product_features["popularity_tier"] = pd.qcut(
    product_features["unique_customers"].rank(method="first"),
    q=5, labels=["Niche", "Low", "Moderate", "Popular", "Best_Seller"], duplicates="drop"
)

print(f"✅ Product features created: {product_features.shape}")
print(product_features.head(3))



 PART 2: PRODUCT PERFORMANCE & CHARACTERISTICS
✅ Product features created: (14950, 21)
   Product_ID  total_revenue  avg_revenue_per_txn  num_transactions  total_units_sold  avg_unit_price  unique_customers  num_stores     first_sale_date      last_sale_date  days_on_market  sales_velocity  Category           Sub_Category  Color  Production_Cost  profit_per_unit  profit_margin_pct  \
0           1       23210.82               145.07               160               168          166.51               150          33 2023-01-01 12:39:00 2023-03-15 00:00:00              73           65.75  Feminine      Coats and Blazers    NaN        23.450001       143.059999              85.92   
1           2       18680.75               112.53               166               190          117.69               158          32 2023-01-01 00:00:00 2023-03-02 00:00:00              61           81.64  Feminine  Sweaters and Knitwear   PINK            24.35            93.34              79.31   
2           

In [ ]:
# PART 3: TEMPORAL & SEASONALITY FEATURES

print("\n" + "="*70)
print(" PART 3: TEMPORAL & SEASONALITY FEATURES")
print("="*70)

# Ensure datetime format
transactions["Date"] = pd.to_datetime(transactions["Date"], errors="coerce")

# 1. Basic calendar features

transactions["year"] = transactions["Date"].dt.year
transactions["month"] = transactions["Date"].dt.month
transactions["day"] = transactions["Date"].dt.day
transactions["weekday"] = transactions["Date"].dt.dayofweek   # 0=Mon, 6=Sun
transactions["weekday_name"] = transactions["Date"].dt.day_name()
transactions["quarter"] = transactions["Date"].dt.quarter
transactions["week_of_year"] = transactions["Date"].dt.isocalendar().week

# 2. Seasonality

season_map = {
    12: "Winter", 1: "Winter", 2: "Winter",
    3: "Spring", 4: "Spring", 5: "Spring",
    6: "Summer", 7: "Summer", 8: "Summer",
    9: "Fall", 10: "Fall", 11: "Fall"
}
transactions["season"] = transactions["month"].map(season_map)

# 3. Business calendar flags

transactions["is_weekend"] = transactions["weekday"].isin([5, 6]).astype(int)
transactions["is_month_start"] = (transactions["day"] <= 5).astype(int)
transactions["is_month_end"] = (transactions["day"] >= 25).astype(int)

# 4. Key holidays (rough estimates)

transactions["is_holiday_season"] = transactions["month"].isin([11, 12]).astype(int)
transactions["is_new_year"] = (transactions["month"] == 1).astype(int)
transactions["is_valentine"] = ((transactions["month"] == 2) & (transactions["day"].between(10, 20))).astype(int)
transactions["is_back_to_school"] = (
    (transactions["month"] == 8) | ((transactions["month"] == 9) & (transactions["day"] <= 15))
).astype(int)

print("✅ Temporal features added:",
      [col for col in transactions.columns if col in [
          "year","month","day","weekday","weekday_name","quarter","week_of_year",
          "season","is_weekend","is_month_start","is_month_end",
          "is_holiday_season","is_new_year","is_valentine","is_back_to_school"
      ]])
print(f"✅ Temporal features created: {transactions.shape}")
print(transactions.head(3))



 PART 3: TEMPORAL & SEASONALITY FEATURES
✅ Temporal features added: ['year', 'month', 'day', 'weekday', 'weekday_name', 'quarter', 'week_of_year', 'season', 'is_weekend', 'is_month_start', 'is_month_end', 'is_holiday_season', 'is_new_year', 'is_valentine', 'is_back_to_school']
✅ Temporal features created: (6284272, 35)
   Transaction_ID           Invoice_ID  Line  Customer_ID  Product_ID Size   Color  Unit_Price  Quantity                Date  Discount  Line_Total  Store_ID  Employee_ID Currency Currency_Symbol                SKU Transaction_Type Payment_Method  Invoice_Total  year  month  day  weekday  \
0               1  INV-US-001-01628272     1        42813        2025    M  YELLOW        36.5         1 2023-01-01 19:46:00       0.4        21.9         1           11      USD               $  CHCO2025-M-YELLOW             Sale    Credit Card           21.9  2023      1    1        6   
1               2  INV-US-001-01628273     1        19756        2794    L  YELLOW        49.5  

In [ ]:
# PART 4: STORE & GEOGRAPHIC FEATURES

print("\n" + "="*70)
print(" PART 4: STORE & GEOGRAPHIC FEATURES")
print("="*70)

# Store size categories
stores['store_size'] = pd.qcut(
    stores['Number_of_Employees'],
    q=3, labels=['Small', 'Medium', 'Large'], duplicates='drop'
)

# Economic indicators by country (simplified but realistic)
country_metrics = {
    'United States': {'gdp_per_capita': 65000, 'fashion_index': 85, 'market_maturity': 'Mature'},
    '中国': {'gdp_per_capita': 12000, 'fashion_index': 78, 'market_maturity': 'Growing'},
    'Deutschland': {'gdp_per_capita': 48000, 'fashion_index': 92, 'market_maturity': 'Mature'},
    'France': {'gdp_per_capita': 43000, 'fashion_index': 95, 'market_maturity': 'Mature'},
    'España': {'gdp_per_capita': 30000, 'fashion_index': 80, 'market_maturity': 'Mature'},
    'United Kingdom': {'gdp_per_capita': 42000, 'fashion_index': 88, 'market_maturity': 'Mature'},
    'Portugal': {'gdp_per_capita': 25000, 'fashion_index': 75, 'market_maturity': 'Growing'}
}

# Add to stores
for country, metrics in country_metrics.items():
    stores.loc[stores['Country'] == country, 'gdp_per_capita'] = metrics['gdp_per_capita']
    stores.loc[stores['Country'] == country, 'fashion_index'] = metrics['fashion_index']
    stores.loc[stores['Country'] == country, 'market_maturity'] = metrics['market_maturity']

# Store performance
store_performance = transactions.groupby('Store_ID').agg({
    'Line_Total': 'sum',
    'Invoice_ID': 'nunique',
    'Customer_ID': 'nunique'
}).round(2)
store_performance.columns = ['store_total_revenue', 'store_total_orders', 'store_unique_customers']

stores = stores.merge(store_performance.reset_index(), on='Store_ID', how='left')
stores['revenue_per_employee'] = (stores['store_total_revenue'] / stores['Number_of_Employees']).round(2)

print(f"✅ Store features created: {stores.shape}")
print(stores.head(3))



 PART 4: STORE & GEOGRAPHIC FEATURES
✅ Store features created: (35, 16)
   Store_ID        Country         City         Store_Name  Number_of_Employees ZIP_Code   Latitude   Longitude store_size  gdp_per_capita  fashion_index market_maturity  store_total_revenue  store_total_orders  store_unique_customers  revenue_per_employee
0         1  United States     New York     Store New York                    8    10001  40.712799  -74.005997      Small         65000.0           85.0          Mature          23791932.49              378882                  107047            2973991.56
1         2  United States  Los Angeles  Store Los Angeles                    7    90001    34.0522 -118.243698      Small         65000.0           85.0          Mature          17966592.86              285543                   89791            2566656.12
2         3  United States      Chicago      Store Chicago                    8    60601  41.878101  -87.629799      Small         65000.0           85.0   

In [ ]:
# PART 5: INTERACTION & PREFERENCE FEATURES (Memory-Safe)

print("\n" + "="*70)
print(" PART 5: INTERACTION & PREFERENCE FEATURES (Optimized)")
print("="*70)

# 1. Customer preferred category (spend-based)
cust_cat_spend = (
    transactions.merge(products[["Product_ID", "Category"]], on="Product_ID")
    .groupby(["Customer_ID", "Category"])["Line_Total"]
    .sum()
    .reset_index()
)

# Only keep the TOP category per customer
customer_pref_category = (
    cust_cat_spend.loc[cust_cat_spend.groupby("Customer_ID")["Line_Total"].idxmax()]
    [["Customer_ID", "Category"]]
    .rename(columns={"Category": "preferred_category"})
)

# 2. Customer preferred store
cust_store_visits = (
    transactions.groupby(["Customer_ID", "Store_ID"])["Invoice_ID"]
    .count()
    .reset_index(name="visit_count")
)

customer_primary_store = (
    cust_store_visits.loc[cust_store_visits.groupby("Customer_ID")["visit_count"].idxmax()]
    [["Customer_ID", "Store_ID"]]
    .rename(columns={"Store_ID": "primary_store_id"})
)

# 3. Price sensitivity (no wide pivot, just aggregates)
customer_price = transactions.groupby("Customer_ID").agg(
    avg_price_point=("Unit_Price", "mean"),
    price_variance=("Unit_Price", "std"),
    avg_discount_value=("Discount", "mean"),
    total_discounts_claimed=("Discount", "sum"),
).round(2)

customer_price["price_sensitivity_score"] = (
    (customer_price["avg_discount_value"] / customer_price["avg_price_point"]) * 100
).round(2)

print(f"✅ Interaction features merged. Final shape: {customer_features.shape}")
print(customer_features.head(3))


 PART 5: INTERACTION & PREFERENCE FEATURES (Optimized)
✅ Interaction features merged. Final shape: (1268571, 36)
             total_orders  total_spent  avg_order_value  spending_std      first_purchase       last_purchase  total_transactions  unique_products_bought  unique_stores_visited  total_discount_used  days_since_last_purchase  days_as_customer  purchase_frequency_per_month recency_score  \
Customer_ID                                                                                                                                                                                                                                                                                     
1                       2       158.28            39.57         32.21 2023-03-09 18:45:00 2023-09-01 17:05:00                   4                       4                      1                  0.9                       570               176                          0.34             1   
2                  

In [ ]:
# ================================================================
# 1️⃣ Prepare Customer Features for Merge
# ================================================================
# Select only relevant customer-level features for transactions
customer_merge_cols = [
    'Customer_ID', 'Age', 'AgeGroup', 'Gender', 'Country',
    'lifecycle_stage', 'customer_value_segment',
    'top_category', 'avg_basket_value', 'purchase_frequency_per_month'
]

customer_features_subset = customer_features.reset_index()[customer_merge_cols]

# ================================================================
# 2️⃣ Merge Customer Features into Transactions
# ================================================================
ml_transactions = transactions.merge(
    customer_features_subset,
    on='Customer_ID',
    how='left'
)

# ================================================================
# 3️⃣ Merge Product Features into Transactions
# ================================================================
product_merge_cols = [
    'Product_ID', 'Category', 'Sub_Category', 'profit_per_unit',
    'profit_margin_pct', 'price_tier', 'popularity_tier'
]

ml_transactions = ml_transactions.merge(
    product_features[product_merge_cols],
    on='Product_ID',
    how='left'
)

# ================================================================
# 4️⃣ Merge Store Features into Transactions
# ================================================================
store_merge_cols = [
    'Store_ID', 'Country', 'City', 'store_size', 'gdp_per_capita',
    'fashion_index', 'revenue_per_employee'
]

ml_transactions = ml_transactions.merge(
    stores[store_merge_cols],
    on='Store_ID',
    how='left',
    suffixes=('_customer', '_store')
)

# ================================================================
# 5️⃣ Sanity Checks
# ================================================================
print("🔎 Merge integrity checks:")
print("Missing Customer_ID:", ml_transactions['Customer_ID'].isna().sum())
print("Missing Product_ID:", ml_transactions['Product_ID'].isna().sum())
print("Missing Store_ID:", ml_transactions['Store_ID'].isna().sum())

print(f"\n✅ Final ML transaction dataset shape: {ml_transactions.shape}")

🔎 Merge integrity checks:
Missing Customer_ID: 0
Missing Product_ID: 0
Missing Store_ID: 0

✅ Final ML transaction dataset shape: (6284272, 56)


In [ ]:
# PART 7: SAVE DATASETS
# ================================================================
print("\n" + "="*70)
print("💾 PART 7: SAVING FEATURE-ENGINEERED DATASETS")
print("="*70)

# Save all feature sets
customer_features.to_csv('features_customers.csv', index=False)
product_features.to_csv('features_products.csv', index=False)
stores.to_csv('features_stores.csv', index=False)
ml_transactions.to_csv('ml_ready_transactions.csv', index=False)

print("✅ Saved 4 feature-engineered datasets:")
print("   1. features_customers.csv - Customer demographics & behavior")
print("   2. features_products.csv - Product performance & characteristics")
print("   3. features_stores.csv - Store operations & geography")
print("   4. ml_ready_transactions.csv - Complete transaction dataset with all features")

# ================================================================
# PART 8: FEATURE SUMMARY
# ================================================================
print("\n" + "="*70)
print("📊 FEATURE ENGINEERING SUMMARY")
print("="*70)

print("\n🎯 CUSTOMER FEATURES (Total columns: {})".format(len(customer_features.columns)))
print("   Demographics: age, age_group, Gender, Country, City, Job Title")
print("   Behavioral: total_orders, total_spent, purchase_frequency, recency")
print("   Segments: lifecycle_stage, customer_value_segment, RFM scores")
print("   Preferences: preferred_category, primary_store_id, price_sensitivity")

print("\n🛍️ PRODUCT FEATURES (Total columns: {})".format(len(product_features.columns)))
print("   Performance: total_revenue, units_sold, sales_velocity")
print("   Pricing: price_tier, profit_margin, avg_unit_price")
print("   Popularity: unique_customers, popularity_tier")
print("   Attributes: Category, Sub Category, Color")

print("\n📅 TEMPORAL FEATURES")
print("   Time: year, month, quarter, week, day_of_week")
print("   Seasonality: season, is_weekend, is_holiday_season")
print("   Events: is_valentine, is_back_to_school, is_month_end")

print("\n🏬 STORE FEATURES (Total columns: {})".format(len(stores.columns)))
print("   Location: Country, City, gdp_per_capita, market_maturity")
print("   Operations: store_size, revenue_per_employee, fashion_index")
print("   Performance: store_total_revenue, store_unique_customers")

print("\n🔗 INTERACTION FEATURES")
print("   Customer-Product: preferred_category")
print("   Customer-Store: primary_store_id, unique_stores_visited")
print("   Price Behavior: price_sensitivity_score, avg_discount_rate")

print("\n" + "="*70)
print("🚀 FEATURE ENGINEERING COMPLETE!")
print("="*70)
print("\n✅ YOU NOW HAVE:")
print("   • {} customer features ready for segmentation & churn prediction".format(len(customer_features.columns)))
print("   • {} product features ready for recommendation systems".format(len(product_features.columns)))
print("   • {} temporal features ready for sales forecasting".format(8))
print("   • {} geographic features ready for expansion analysis".format(3))
print("   • Complete ML dataset with {} rows and {} columns".format(len(ml_transactions), len(ml_transactions.columns)))

print("\n💡 NEXT STEPS - Choose Your ML Model:")
print("   1. Customer Churn Prediction (using lifecycle_stage + RFM)")
print("   2. Sales Forecasting (using temporal + store + product features)")
print("   3. Customer Segmentation (using all customer features)")
print("   4. Product Recommendation (using preferred_category + purchase history)")
print("   5. Dynamic Pricing (using price_sensitivity + market features)")

print("\n🎯 Ready to build impressive ML models that drive business decisions!")


💾 PART 7: SAVING FEATURE-ENGINEERED DATASETS
✅ Saved 4 feature-engineered datasets:
   1. features_customers.csv - Customer demographics & behavior
   2. features_products.csv - Product performance & characteristics
   3. features_stores.csv - Store operations & geography
   4. ml_ready_transactions.csv - Complete transaction dataset with all features

📊 FEATURE ENGINEERING SUMMARY

🎯 CUSTOMER FEATURES (Total columns: 36)
   Demographics: age, age_group, Gender, Country, City, Job Title
   Behavioral: total_orders, total_spent, purchase_frequency, recency
   Segments: lifecycle_stage, customer_value_segment, RFM scores
   Preferences: preferred_category, primary_store_id, price_sensitivity

🛍️ PRODUCT FEATURES (Total columns: 21)
   Performance: total_revenue, units_sold, sales_velocity
   Pricing: price_tier, profit_margin, avg_unit_price
   Popularity: unique_customers, popularity_tier
   Attributes: Category, Sub Category, Color

📅 TEMPORAL FEATURES
   Time: year, month, quarter, w

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

# Define the path where you want to save the files in your Google Drive
drive_path = '/content/drive/MyDrive/feature_engineered_data'

# Create the directory if it doesn't exist
os.makedirs(drive_path, exist_ok=True)

# List of files to save
files_to_save = [
    'features_customers.csv',
    'features_products.csv',
    'features_stores.csv',
    'ml_ready_transactions.csv'
]

print(f"Saving files to {drive_path}...")

for filename in files_to_save:
    source_path = filename
    destination_path = os.path.join(drive_path, filename)
    # Check if the file exists in the current directory before copying
    if os.path.exists(source_path):
        !cp "{source_path}" "{destination_path}"
        print(f"✅ Saved {filename} to Google Drive.")
    else:
        print(f"⚠️ Warning: {filename} not found in current directory. Skipping.")

print("\nAll specified files have been copied to your Google Drive.")

Saving files to /content/drive/MyDrive/feature_engineered_data...
✅ Saved features_customers.csv to Google Drive.
✅ Saved features_products.csv to Google Drive.
✅ Saved features_stores.csv to Google Drive.
✅ Saved ml_ready_transactions.csv to Google Drive.

All specified files have been copied to your Google Drive.
